In [1]:
import keras
import numpy as np
import pandas as pd
import os
from keras.preprocessing import image
import tensorflow as tf
from PIL import Image
from sklearn.metrics import accuracy_score
import re


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input, EfficientNetB0

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.preprocessing import image

In [5]:
df=pd.read_csv('df_f.csv')
df

,Image,Category
0,battery/battery1.jpg,0
1,battery/battery10.jpg,0
2,battery/battery100.jpg,0
3,battery/battery101.jpg,0
4,battery/battery102.jpg,0
...,...,...
15510,white-glass/white-glass95.jpg,12
15511,white-glass/white-glass96.jpg,12
15512,white-glass/white-glass97.jpg,12
15513,white-glass/white-glass98.jpg,12


In [6]:
garbage_types_labels = {
    0: 'battery',
    1: 'biological',
    3: 'brown-glass',
    4: 'cardboard',
    5: 'clothes',
    6: 'green-glass',
    7: 'metal',
    8: 'paper',
    9: 'plastic',
    10: 'shoes',
    11: 'trash',
    12: 'white-glass'
}

In [7]:
df

,Image,Category
0,battery/battery1.jpg,0
1,battery/battery10.jpg,0
2,battery/battery100.jpg,0
3,battery/battery101.jpg,0
4,battery/battery102.jpg,0
...,...,...
15510,white-glass/white-glass95.jpg,12
15511,white-glass/white-glass96.jpg,12
15512,white-glass/white-glass97.jpg,12
15513,white-glass/white-glass98.jpg,12


In [8]:
df["Category"] = df["Category"].replace(garbage_types_labels) 

In [9]:
df

,Image,Category
0,battery/battery1.jpg,battery
1,battery/battery10.jpg,battery
2,battery/battery100.jpg,battery
3,battery/battery101.jpg,battery
4,battery/battery102.jpg,battery
...,...,...
15510,white-glass/white-glass95.jpg,white-glass
15511,white-glass/white-glass96.jpg,white-glass
15512,white-glass/white-glass97.jpg,white-glass
15513,white-glass/white-glass98.jpg,white-glass


### TO CSV

In [10]:
data_directory = "garbage_classification/"

In [11]:
from tensorflow.keras.applications import xception
from tensorflow.keras.layers import Lambda, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

def create_xception_model(img_width, img_height, channels, num_classes):
    # Cargar el modelo Xception preentrenado
    xception_layer = xception.Xception(include_top=False, input_shape=(img_width, img_height, channels),
                                      weights='imagenet')

    # No queremos entrenar los pesos importados
    xception_layer.trainable = False

    model = Sequential()
    model.add(keras.Input(shape=(img_width, img_height, channels)))

    # Crear una capa personalizada para aplicar el preprocesamiento
    def xception_preprocessing(img):
        return xception.preprocess_input(img)

    model.add(Lambda(xception_preprocessing))

    model.add(xception_layer)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

    model.summary()

    return model


In [12]:
model=create_xception_model(224,224,3,12)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 12)                24588     
                                                                 
Total params: 20,886,068
Trainable params: 24,588
Non-trainable params: 20,861,480
_________________________________________________________________


In [18]:
img_height = 224
img_width = 224
img_size=(img_width,img_height)
channels=3


In [19]:

# We first split the data into two sets and then split the validate_df to two sets
train_df, validate_df = train_test_split(df, test_size=0.2, random_state=42)
validate_df, test_df = train_test_split(validate_df, test_size=0.3, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train = train_df.shape[0]
validate = validate_df.shape[0]

print('train size = ', validate , 'validate size = ', validate, 'test size = ', test_df.shape[0])

train size =  2172 validate size =  2172 test size =  931


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size=64

train_datagen = ImageDataGenerator(
    
    ###  Augmentation Start  ###
    
    rotation_range=30,
    shear_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip = True,
    width_shift_range=0.2,
    height_shift_range=0.2
    
    ##  Augmentation End  ###
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    data_directory, 
    x_col='Image',
    y_col='Category',
    target_size=img_size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 12412 validated image filenames belonging to 12 classes.


In [21]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    data_directory, 
    x_col='Image',
    y_col='Category',
    target_size=img_size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 931 validated image filenames belonging to 12 classes.


In [22]:
validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    data_directory, 
    x_col='Image',
    y_col='Category',
    target_size=img_size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 2172 validated image filenames belonging to 12 classes.


In [23]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv',index=False)
validate_df.to_csv('validation.csv',index=False)

In [25]:
print(train,test_df.shape[0])

12412 931


In [26]:
def create_train_generator(train_df, data_directory, img_size, batch_size):
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=data_directory,
        x_col='Image',
        y_col='Category',
        target_size=img_size,
        class_mode='categorical',
        batch_size=batch_size
    )
    return train_generator

def create_validation_generator(validate_df, data_directory, img_size, batch_size):
    validation_datagen = ImageDataGenerator(rescale=1./255)
    validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=validate_df,
        directory=data_directory,
        x_col='Image',
        y_col='Category',
        target_size=img_size,
        class_mode='categorical',
        batch_size=batch_size
    )
    return validation_generator

def create_test_generator(test_df, data_directory, img_size, batch_size):
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=data_directory,
        x_col='Image',
        y_col='Category',
        target_size=img_size,
        class_mode='categorical',
        batch_size=batch_size
    )
    return test_generator

In [27]:
img_size = (224, 224)
batch_size = 32

# Crear generadores de datos
train_generator = create_train_generator(train_df, data_directory, img_size, batch_size)
validation_generator = create_validation_generator(validate_df, data_directory, img_size, batch_size)
test_generator = create_test_generator(test_df, data_directory, img_size, batch_size)

Found 12412 validated image filenames belonging to 12 classes.
Found 2172 validated image filenames belonging to 12 classes.
Found 931 validated image filenames belonging to 12 classes.


In [28]:
from keras.callbacks import EarlyStopping

In [29]:
early_stop = EarlyStopping(patience = 2, verbose = 1, monitor='val_categorical_accuracy' , mode='max', min_delta=0.001, restore_best_weights = True)

callbacks = [early_stop]

print('call back defined!')

call back defined!


In [30]:
from datetime import datetime

def train_model(model,train_generator, validation_generator, total_train, validate, batch_size, epochs, callbacks, path, name):
    start = datetime.now()

    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validate // batch_size,
        steps_per_epoch=train // batch_size,
        callbacks=callbacks
    )

    try:
        model.save(f"{path}/{name}.h5")
    except:
        print("Modelo no guardado")

    end = datetime.now()
    print(f"It took {end-start} time")

    return history


In [31]:
history = train_model(model,train_generator, validation_generator, train, validate, batch_size, epochs, callbacks, ".", "last_model0306")

387/387 [==============================] - 1745s 4s/step - loss: 0.4483 - categorical_accuracy: 0.8742 - val_loss: 2.6214 - val_categorical_accuracy: 0.0378
It took 0:29:06.000715 time
